In [1]:
# Retrieve DHIS2 data for a specified time range
# using Python wrapper and export to file.
# by Ali Tobah - atobah@gmail.com

# Resources:
# https://pypi.org/project/dhis2.py/
# https://docs.dhis2.org/en/develop/using-the-api/dhis-core-version-240/introduction.html
# https://play.dhis2.org/40.2.0/api/resources

# Only need to run this once for each session.
!pip install dhis2.py

In [2]:
import pandas as pd
import getpass
from dhis2 import Api

In [3]:
def retrieveNameID(dataParam):
    '''
        Input: Parameter for the API request.
        Output: Prints DF shape and head.
        Return: Dataframe.
    '''
    # Run a request on an API object to get data parameter names
    # and IDs. The request returns a dictionary of one key with a
    # list of dictionaries as its value.
    try:
        response = api.get_paged(dataParam, merge=True)
    except:
        outString = 'Request for parameter "' + dataParam + '" failed.'
        print(outString)
        # Will return None
        return

    # Read the data and feed into a dataframe
    dhisData = response[dataParam]
    dataDF = pd.DataFrame(dhisData)

    # Rename columns to unique names and print
    colName = dataParam[:-1] + 'Name'
    colID = dataParam[:-1] + 'ID'
    dataDF = dataDF.rename(columns={'displayName': colName, 'id': colID})

    # Print dataframe info for user visual check
    print("Dataframe shape: ", dataDF.shape, '\n')
    print(dataDF.head())

    return dataDF

In [4]:
# Create an API object for the demo data.

# Prompt for URL where data is located.
# For demo data, should use "https://play.dhis2.org/demo".
# NOTE: Sometimes this is not available, and so the API request
# doesn't return data. In this case, should use "https://play.dhis2.org/2.39.4.1".
print("Demo data location is 'https://play.dhis2.org/demo'.")
print("However, this is not always available.")
print("If the API call doesn't return data,\nuse 'https://play.dhis2.org/2.39.4.1'.\n")
dhisURL = input("Data location (URL): ")

# Prompt for username and password while creating the object.
# User name is "admin" for demo data
# Password is "district" for demo data. Use getpass to protect it.
api = Api(dhisURL,
          input("Username (Use 'admin' for demo data): "),
          getpass.getpass(prompt="Password (Use 'district' for demo data): "))

Demo data location is 'https://play.dhis2.org/demo'.
However, this is not always available.
If the API call doesn't return data,
use 'https://play.dhis2.org/2.39.4.1'.

Data location (URL): https://play.dhis2.org/demo
Username (Use 'admin' for demo data): admin
Password (Use 'district' for demo data): ··········


In [5]:
# Get dataset names and IDs dataframe.
dataSetDF = retrieveNameID('dataSets')

# If returned value is not a df, the request failed
if not isinstance(dataSetDF, pd.DataFrame):
    print('Error occurred while retrieving dataset names and IDs.')
else:
    # Create a list for use in the request.
    dataSetIDList = list(dataSetDF.dataSetID)

Dataframe shape:  (26, 2) 

                      dataSetName    dataSetID
0             ART monthly summary  lyLU2wR22tC
1                    Child Health  BfMAe6Itzgt
2  Clinical Monitoring Checklist   VTdjfLXXmoi
3                       EPI Stock  TuL8IOPzpHh
4              Emergency Response  Lpw6GcnTrmS


In [6]:
# Get organisational unit names and IDs dataframe.
orgUnitDF = retrieveNameID('organisationUnits')

# If returned value is not a df, the request failed
if not isinstance(orgUnitDF, pd.DataFrame):
    print('Error occurred while retrieving organisational unit names and IDs.')
else:
    # Create a list for use in the request.
    orgUnitIDList = list(orgUnitDF.organisationUnitID)
    # Create a dictionary to map IDs to names in final result.
    orgUnitDict = orgUnitDF.set_index('organisationUnitID')['organisationUnitName'].to_dict()

Dataframe shape:  (2684, 2) 

  organisationUnitName organisationUnitID
0         Panderu MCHP        ueuQlqb8ccl
1    A / Kitily Kebele        TAppgqrqMeP
2         Abach Kebele        mNRukTCxLRr
3         Abadi Kebele        Zxfz9mWpFK9
4  Abado Bukesa Kebele        uF97jQcna9t


In [7]:
# Get data element names and IDs dataframe.
dataElementDF = retrieveNameID('dataElements')

# If returned value is not a df, the request failed
if not isinstance(dataElementDF, pd.DataFrame):
    print('Error occurred while retrieving data element names and IDs.')
else:
    # Create a dictionary to map IDs to names in final result.
    dataElementDict = dataElementDF.set_index('dataElementID')['dataElementName'].to_dict()

Dataframe shape:  (1041, 2) 

          dataElementName dataElementID
0           ANC 1st visit   fbfJHSPpUQD
1           ANC 2nd visit   cYeuwXTCPkU
2       ANC 3 - Predicted   UKlgiJaKrL3
3           ANC 3rd visit   Jtf34kNZhzP
4  ANC 4th or more visits   hfdmMSPBgLG


In [8]:
# Get category option names and IDs dataframe.
categoryOptiontDF = retrieveNameID('categoryOptionCombos')

# If returned value is not a df, the request failed
if not isinstance(categoryOptiontDF, pd.DataFrame):
    print('Error occurred while retrieving category option names and IDs.')
else:
    # Create a dictionary to map IDs to names in final result.
    categOptDict = categoryOptiontDF.set_index('categoryOptionComboID')['categoryOptionComboName'].to_dict()

Dataframe shape:  (231, 2) 

  categoryOptionComboName categoryOptionComboID
0                   0-11m           S34ULMcHMca
1                   0-11m           sqGRzCziswD
2                    0-4y           o2gxEt6Ek2C
3                  12-59m           LEDQQXEpWUl
4                  12-59m           wHBMVthqIX4


In [9]:
# Prompt for start and end dates.
# When testing, use 2022-01-01 and 2022-01-31
startDate = input("Start date (Suggest '2022-01-01' for demo data): ")
endDate = input("End date (Suggest '2022-01-31' for demo data): ")

Start date (Suggest '2022-01-01' for demo data): 2022-01-01
End date (Suggest '2022-01-31' for demo data): 2022-01-31


In [10]:
# Request data for all datasets in dataset list,
# and all organisational units in organisational unit list,
# for given start and end dates.

# Since data is too large for one request, break organisational
# unit ID list into a list of lists of organisational units, each
# of size 100.
orgNestList = [orgUnitIDList[i:i + 100] for i in range(0, len(orgUnitIDList), 100)]

# Create an empty list to hold the responses.
# This will be a list of dictionaries, each dictionary
# representing one data element, as seen in the next cell.
responseList = []
i=0    # List counter for user convenience
print("Expect", len(orgNestList), "iterations:")

# Retrieve data for each 100 orgUnits and add to
# response list using extend (as opposed to append).
for eaOrgList in orgNestList:
    response = api.get('dataValueSets', params={
        'dataSet': dataSetIDList,
        'orgUnit': eaOrgList,
        'startDate': startDate,
        'endDate': endDate
    })
    # Try to extract data from returned object
    try:
        addnlElements = response.json()['dataValues']
    except:
        print("Request failed.")
        break

    responseList.extend(addnlElements)

    # Increment counter and print for user patience
    i+=1
    print(i, "..", len(addnlElements), "data elements")


print("Total data elements:", len(responseList))


Expect 27 iterations:
1 .. 0 data elements
2 .. 116 data elements
3 .. 331 data elements
4 .. 388 data elements
5 .. 306 data elements
6 .. 197 data elements
7 .. 262 data elements
8 .. 497 data elements
9 .. 396 data elements
10 .. 243 data elements
11 .. 283 data elements
12 .. 414 data elements
13 .. 229 data elements
14 .. 314 data elements
15 .. 388 data elements
16 .. 96 data elements
17 .. 734 data elements
18 .. 62 data elements
19 .. 715 data elements
20 .. 279 data elements
21 .. 620 data elements
22 .. 632 data elements
23 .. 270 data elements
24 .. 489 data elements
25 .. 404 data elements
26 .. 292 data elements
27 .. 323 data elements
Total data elements: 9280


In [11]:
# Print first value in the list (i.e the first data element)
# for visual confirmation, but only if some data was returned.
if len(responseList) > 0:
    cellOutput = responseList[0]
else:
    cellOutput = "No data returned."

cellOutput

{'dataElement': 's46m5MS0hxu',
 'period': '202201',
 'orgUnit': 'yMCshbaVExv',
 'categoryOptionCombo': 'Prlt0C1RF0s',
 'attributeOptionCombo': 'HllvX50cXC0',
 'value': '11',
 'storedBy': 'anaam',
 'created': '2022-09-05T13:06:21.000+0000',
 'lastUpdated': '2022-05-29T22:06:10.000+0000',
 'comment': None,
 'followup': False}

In [12]:
# Feed all data elements from the list into a dataframe.
# But only if some data was returned.
if len(responseList) > 0:
    dhisDF = pd.DataFrame(responseList)
    print("Dataframe shape: ", dhisDF.shape, '\n')
    cellOutput = dhisDF.head()
else:
    cellOutput = "No data returned."

cellOutput

Dataframe shape:  (9280, 11) 



,dataElement,period,orgUnit,categoryOptionCombo,attributeOptionCombo,value,storedBy,created,lastUpdated,comment,followup
0,s46m5MS0hxu,202201,yMCshbaVExv,Prlt0C1RF0s,HllvX50cXC0,11,anaam,2022-09-05T13:06:21.000+0000,2022-05-29T22:06:10.000+0000,None,False
1,s46m5MS0hxu,202201,yMCshbaVExv,V6L425pT3A0,HllvX50cXC0,13,anaam,2022-09-05T13:06:21.000+0000,2022-05-29T22:06:10.000+0000,None,False
2,UOlfIjgN8X6,202201,yMCshbaVExv,Prlt0C1RF0s,HllvX50cXC0,17,anaam,2022-09-05T13:06:21.000+0000,2022-05-29T22:06:10.000+0000,None,False
3,UOlfIjgN8X6,202201,yMCshbaVExv,V6L425pT3A0,HllvX50cXC0,9,anaam,2022-09-05T13:06:21.000+0000,2022-05-29T22:06:10.000+0000,None,False
4,z7duEFcpApd,202201,yMCshbaVExv,Prlt0C1RF0s,HllvX50cXC0,10,anaam,2022-09-05T13:06:21.000+0000,2022-05-29T22:06:10.000+0000,None,False


In [13]:
# Replace IDs with names using dictionaries from above.
# Note that the attributeOptionCombo uses the same
# dictionary as categoryOptionCombo. This is not a typo.
# Do this only if some data was returned.
if len(responseList) > 0:
    dhisDF = dhisDF.copy()
    dhisDF['dataElement'] = dhisDF['dataElement'].map(dataElementDict)
    dhisDF['orgUnit'] = dhisDF['orgUnit'].map(orgUnitDict)
    dhisDF['categoryOptionCombo'] = dhisDF['categoryOptionCombo'].map(categOptDict)      # Same dictionary.
    dhisDF['attributeOptionCombo'] = dhisDF['attributeOptionCombo'].map(categOptDict)    # Not a typo.
    print("Dataframe shape: ", dhisDF.shape, '\n')
    cellOutput = dhisDF.head()
else:
    cellOutput = "No data returned."

cellOutput

Dataframe shape:  (9280, 11) 



,dataElement,period,orgUnit,categoryOptionCombo,attributeOptionCombo,value,storedBy,created,lastUpdated,comment,followup
0,BCG doses given,202201,Babara CHC,"Fixed, <1y",default,11,anaam,2022-09-05T13:06:21.000+0000,2022-05-29T22:06:10.000+0000,None,False
1,BCG doses given,202201,Babara CHC,"Outreach, <1y",default,13,anaam,2022-09-05T13:06:21.000+0000,2022-05-29T22:06:10.000+0000,None,False
2,Fully Immunized child,202201,Babara CHC,"Fixed, <1y",default,17,anaam,2022-09-05T13:06:21.000+0000,2022-05-29T22:06:10.000+0000,None,False
3,Fully Immunized child,202201,Babara CHC,"Outreach, <1y",default,9,anaam,2022-09-05T13:06:21.000+0000,2022-05-29T22:06:10.000+0000,None,False
4,LLITN given at Penta3,202201,Babara CHC,"Fixed, <1y",default,10,anaam,2022-09-05T13:06:21.000+0000,2022-05-29T22:06:10.000+0000,None,False


In [15]:
# Request file type and export.
# CSV, Excel or no export.
# But only if some data was returned.

if len(responseList) > 0:
    print("Export to CSV, Excel or no export.")
    outFileType = input("csv, excel, nox: ")
    while outFileType not in ['csv', 'excel', 'nox']:
        outFileType = input("Choose from: csv, excel or nox: ")

    newStartDate = startDate.replace('-', '')
    newEndDate = endDate.replace('-', '')
    dhisFileName = "DHIS-Demo-Data" + "-" + newStartDate + "-" + newEndDate

    if outFileType == 'csv' :
        dhisFileName = dhisFileName + '.csv'
        print("Exporting ...")
        dhisDF.to_csv(dhisFileName)
        print("Data exported to:", dhisFileName)

    elif outFileType == 'excel' :
        dhisFileName = dhisFileName + '.xlsx'
        print("Exporting ...")
        dhisDF.to_excel(dhisFileName)
        print("Data exported to:", dhisFileName)

    elif outFileType == 'nox' :
        print("Data not exported.")

else:
    print('"No data returned."')

Export to CSV, Excel or no export.
csv, excel, nox: excel
Exporting ...
Data exported to: DHIS-Demo-Data-20220101-20220131.xlsx
